In [58]:
PARAMS_EXTRACT = {'SAMPLE_RATE': 44000,
                 'LOW_FREQ': 500,
                 'HIGH_FREQ': 8000,
                 'BUTTER_ORDER': 1,
                 'AUDIO_DURATION': 60,
                 'CHUNK_DURATION': 10,
                 'OVLP': 1,
                 'MODE_RMBCKG': 'median',
                 'N_RUNNING_MEAN': 10,
                 'NFFT': 1024,
                 'MASK_PARAM1': 26,
                 'MASK_PARAM2': 10,
                 'MAX_RATIO_YX': 7,
                 'MIN_DURATION': 0.2,
                 'MARGIN_T_LEFT': 0.2,
                 'MARGIN_T_RIGHT': 0.2,
                 'MARGIN_F_TOP': 250,
                 'MARGIN_F_BOTTOM': 250,
                 'MARGIN_T': 0.1,
                 'MARGIN_F': 250,
                 'FILTER_ORDER': 5,
                 'SHAPE_RES': "high"}

In [59]:
import os
from pathlib import Path
import maad
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [60]:
# audio package
import librosa

import seaborn as sns

In [61]:
from skimage.transform import resize

# Scikit-Maad (ecoacoustics functions) package
import maad

In [62]:
from maad.util import mean_dB, add_dB, power2dB, dB2power,plot2d, format_features, overlay_rois,plot_shape,plot_spectrum
from scipy import ndimage 
from skimage.morphology import closing
from skimage import measure
from maad import sound, features, rois

In [63]:
def _centroid_features(Sxx, rois=None, im_rois=None): 
    if type(Sxx) is not np.ndarray and len(Sxx.shape) != 2: 
        raise TypeError('Sxx must be an numpy 2D array')  
        
# Convert the spectrogram to a linear scale.
# This step ensures accurate retrieval of the 90th percentile energy within each bounding box.
# If working with a cleaned spectrogram, this directly relates to the SNR
    Sxx = maad.util.dB2power(Sxx)
     
    # check rois 
    if rois is not None: 
        if not(('min_t' and 'min_f' and 'max_t' and 'max_f') in rois): 
            raise TypeError('Array must be a Pandas DataFrame with column names: min_t, min_f, max_t, max_f. ') 
     
    centroid=[] 
    area = []   
    snr = []
    if rois is None: 
        centroid = ndimage.center_of_mass(Sxx) 
        centroid = pd.DataFrame(np.asarray(centroid)).T 
        centroid.columns = ['centroid_y', 'centroid_x'] 
        centroid['area_xy'] = Sxx.shape[0] * Sxx.shape[1]
        centroid['duration_x'] = Sxx.shape[1]
        centroid['bandwidth_y'] = Sxx.shape[0]
        # centroid['snr'] = np.percentile(Sxx, 0.99)
        centroid['snr'] = mean_dB(add_dB(Sxx,axis=0))
    else: 
        if im_rois is not None : 
            # real centroid and area
            rprops = measure.regionprops(im_rois, intensity_image=Sxx)
            centroid = [roi.weighted_centroid for roi in rprops]
            area = [roi.area for roi in rprops]
            # snr = [power2dB(np.percentile(roi.image_intensity,99)) for roi in rprops]
            snr = [power2dB(np.mean(np.sum(roi.image_intensity,axis=0))) for roi in rprops]
        else:
            # rectangular area (overestimation) 
            area = (rois.max_y - rois.min_y) * (rois.max_x - rois.min_x)  
            # centroid of rectangular roi
            for _, row in rois.iterrows() : 
                row = pd.DataFrame(row).T 
                im_blobs = maad.rois.rois_to_imblobs(np.zeros(Sxx.shape), row)     
                rprops = measure.regionprops(im_blobs, intensity_image=Sxx)
                centroid.append(rprops.pop().weighted_centroid) 
               
                snr.append(power2dB(np.mean(np.sum(rprops.pop().image_intensity,axis=0)))) 
                
        centroid = pd.DataFrame(centroid, columns=['centroid_y', 'centroid_x'], index=rois.index)
        
        #99th percentile energy of bbox (99th percentile of the bbox)
        centroid['snr'] = snr
        # duration of the roi represnted in number of pixels 
        centroid['duration_x'] = (rois.max_x -rois.min_x)  
        # duration of the roi represnted in number of pixels 
        centroid['bandwidth_y'] = (rois.max_y -rois.min_y) 
        # area of the spectogram
        centroid['area_xy'] = area      
     
        # merge both the dataframes 
        centroid = rois.join(pd.DataFrame(centroid, index=rois.index))  

    return centroid 

In [64]:
def _select_rois(im_bin, 
                 min_roi=None,
                 max_roi=None, 
                 margins=(0,0), 
                 verbose=True, 
                 display=False, 
                 **kwargs): 
    if max_roi is None:  
        # the maximum ROI is set to the aera of the image for easier processing
        max_roi=im_bin.shape[0]*im_bin.shape[1] 
         
    if min_roi is None: 
        # the min ROI area is set to 1 pixel 
        min_roi = 1 
    
    if verbose :
        print(72 * '_') 

    # merge ROIS
    if sum(margins) !=0 :
        footprint = np.ones((margins[0]*2+1,margins[1]*2+1))
        im_bin = closing(im_bin, footprint)            
    #Finding connected components in binary image 
    labels = measure.label(im_bin)    
    rprops = measure.regionprops(labels) 
     
    rois_bbox = [] 
    rois_label = [] 
     
    for roi in rprops: 
         
        # select the rois  depending on their size 
        if (roi.area >= min_roi) & (roi.area <= max_roi): 
            # gettting the label 
            rois_label.append(roi.label) 
            # append the rectangle coordinates            
            rois_bbox.append (roi.bbox)     
                 
    im_rois = np.isin(labels, rois_label)   
     
    # create a list with labelID and labelName (None in this case) 
    rois_label = list(zip(rois_label,['unknown']*len(rois_label))) 
    
    # check there is ROI
    if len(rois_label)>0 :
        # create a dataframe rois containing the coordonates and the label 
        rois = np.concatenate((np.asarray(rois_label), np.asarray(rois_bbox)), axis=1) 
        rois = pd.DataFrame(rois, columns = ['labelID', 'label', 'min_y','min_x','max_y', 'max_x']) 
        # type conversion to integer
        rois = rois.astype({'label': str,'min_y':int,'min_x':int,'max_y':int, 'max_x':int}) 
        #  half-open interval of bbox from skimage is compensated 
        rois.max_y -= 1 
        rois.max_x -= 1 
        
    else :
        rois = []    
        rois = pd.DataFrame(rois, columns = ['labelID', 'label', 'min_y','min_x','max_y', 'max_x']) 
        rois = rois.astype({'label': str,'min_y':int,'min_x':int,'max_y':int, 'max_x':int}) 
     
  
    if display :  
        ylabel =kwargs.pop('ylabel','Frequency [Hz]') 
        xlabel =kwargs.pop('xlabel','Time [sec]')  
        title  =kwargs.pop('title','Selected ROIs')  
        extent=kwargs.pop('extent',None)
                
        if extent is None : 
            xlabel = 'pseudotime [points]'
            ylabel = 'pseudofrequency [points]'
          
        cmap   =kwargs.pop('cmap','tab20') 
         
        _, fig = maad.util.plot2d (
                                im_rois, 
                                extent = extent,
                                title  = title,  
                                ylabel = ylabel, 
                                xlabel = xlabel,
                                cmap   = cmap, 
                                **kwargs) 
            
    return im_rois, rois 

In [65]:
def extract_rois_full_sig(
    sig,
    params=PARAMS_EXTRACT,
    display=False,
    verbose=False,
    **kwargs):
    # 1. compute the spectrogram
    Sxx, tn, fn, ext = maad.sound.spectrogram(
                                sig,
                                params["SAMPLE_RATE"],
                                nperseg=params["NFFT"],
                                noverlap=params["NFFT"] // 2,
                                flims=[params["LOW_FREQ"], params["HIGH_FREQ"]])

    t_resolution = tn[1] - tn[0]
    f_resolution = fn[1] - fn[0]

    if verbose:
        print("time resolution {}s".format(t_resolution))
        print("frequency resolution {}s".format(f_resolution))

    if display:
        # creating grid for subplots
        fig = plt.figure()
        fig.set_figheight(5)
        fig.set_figwidth(13)
        ax0 = plt.subplot2grid(shape=(2, 4), loc=(0, 0), colspan=1)
        ax1 = plt.subplot2grid(shape=(2, 4), loc=(1, 0), colspan=1)
        ax2 = plt.subplot2grid(shape=(2, 4), loc=(0, 1), colspan=1)
        ax3 = plt.subplot2grid(shape=(2, 4), loc=(1, 1), colspan=1)
        ax4 = plt.subplot2grid(shape=(2, 4), loc=(0, 2), rowspan=2, colspan=2)

        maad.util.plot_wave(sig, fs=params["SAMPLE_RATE"], ax=ax0)

        maad.util.plot_spectrogram(
            Sxx,
            extent=ext,
            ax=ax1,
            title="1. original spectrogram",
            interpolation=None,
            now=False)
        
    # 3. convert to dB
    #  adding 96 so that every value is positive
    Sxx_dB = maad.util.power2dB(Sxx, db_range=96) + 96
    
    # 2. Clean spectrogram : remove background)
    Sxx_clean_dB, noise_profile = maad.sound.remove_background_along_axis(Sxx_dB,
                                                      mode=params["MODE_RMBCKG"],
                                                      N=params["N_RUNNING_MEAN"],
                                                      display=False)

    if display:
        maad.util.plot_spectrogram(
            Sxx_clean_dB,
            extent=ext,
            log_scale=False,
            ax=ax2,
            title="2. cleaned spectrogram",
            interpolation='none',
            now=False,
            vmin = 0,
            vmax = np.percentile(Sxx_clean_dB,99.9)
        )
    
    # 4. estimate SNR to set a threshold for the spectogram
    _,bgn,snr,_,_,_ = maad.sound.spectral_snr(maad.util.dB2power(Sxx_clean_dB))
    if verbose :
        print('BGN {}dB / SNR {}dB'.format(bgn,snr))
        
    # 5. Convert the spectrogram to binary to identify segments with 
    # acoustic activity. Adjust parameters as needed to capture more or 
    # fewer Regions of Interest (ROIs) of varying sizes.        
    
    im_mask = maad.rois.create_mask(
        Sxx_clean_dB,
        mode_bin="absolute",
        bin_h=  params["MASK_PARAM1"],
        bin_l=  params["MASK_PARAM2"]
    )

    if display:
        maad.util.plot_spectrogram(
            im_mask,
            extent=ext,
            ax=ax3,
            title="3. mask",
            interpolation=None,
            now=True,
        )

    # 6. a. Retrieve the mask containing regions of interest (ROIs) - "im_rois",
    # the bounding box for each ROI - "rois_bbox", and a unique identifier for each ROI.
    # This data is stored in the pandas dataframe "rois"
    margins = (round(params["MARGIN_F_BOTTOM"] / f_resolution),
               round(params["MARGIN_T_LEFT"] / t_resolution)) 
    im_rois, df_rois = _select_rois(im_mask, min_roi=None, margins = margins)
    
    # format it to initial resolution of frequency and time
    df_rois = maad.util.format_features(df_rois, tn, fn)

    # . find the centroid id and adding the centroid feaures ('centroid_y',
    # 'centroid_x', 'duration_x', 'bandwidth_y', 'area_xy') to the df_rois
    df_rois = _centroid_features(Sxx_clean_dB, df_rois, im_rois)
    
    # format it to initial resolution of frequency and time
    df_rois = maad.util.format_features(df_rois, tn, fn)
    
    # test whether any ROI presnt
    if len(df_rois) > 0:    
        
        # 7. Remove invalid ROI
        df_rois = df_rois[df_rois.min_x < df_rois.max_x]
        df_rois = df_rois[df_rois.min_y < df_rois.max_y]
        
       
        
        # check the ratio x/y
        df_rois['ratio_yx'] = (df_rois.max_y -df_rois.min_y) / (df_rois.max_x -df_rois.min_x) 
        if params["MAX_RATIO_YX"] is not None :
            df_rois = df_rois[df_rois['ratio_yx'] < params["MAX_RATIO_YX"]] 
    
        # Drop columns which are not needed
        df_rois = df_rois.drop(columns=["labelID", "label"])

        # only events longer than MINduration are selected
        df_rois = df_rois[((df_rois["max_t"]-df_rois["min_t"])>params["MIN_DURATION"])]

        if verbose:
            print("=> AFTER MERGING FOUND {} ROIS".format(len(df_rois)))
        
        if display:
            #  dB conversion
            X = maad.util.power2dB(Sxx, db_range=96) + 96
            kwargs.update({"vmax": np.max(X)})
            kwargs.update({"vmin": np.min(X)})
            kwargs.update({"extent": ext})
            kwargs.update({"figsize": (1, 2.5)})
            maad.util.plot_spectrogram(
                X, ext, log_scale=False, ax=ax4, title="5. Overlay ROIs"
            )
            maad.util.overlay_rois(X, df_rois,
                              edge_color='yellow',
                              ax=ax4, fig=fig, **kwargs)
            kwargs.update(
                {"ms": 4, "marker": "+", "fig": fig, "ax": ax4})
            
            fig.suptitle(kwargs.pop("suptitle", ""))
            fig.tight_layout()
    # create the shape of the rois and store it in a dataframe
    df_rois_for_shape = df_rois[["min_y", "min_x", "max_y", "max_x"]]
    # check whether we have any rois in the audio  
    if not df_rois_for_shape.empty:   
        # apply 2d gabor filter over the rois and extract features from them using the inbuilt maad function
        df_shape, params_shape = maad.features.shape_features(Sxx_clean_dB, 
                                                            resolution=params["SHAPE_RES"], 
                                                            rois=df_rois_for_shape)
        # format the features according to the spectogram received
        df_shape = maad.util.format_features(df_shape, tn, fn)
        # maad.util.plot_shape(df_shape, params_shape)
        combined_df_features = pd.concat([df_shape, df_rois], axis=1)
        combined_df_features = combined_df_features.loc[:, ~combined_df_features.columns.duplicated()]
        
        return combined_df_features

In [66]:

# 1. specify the file path
excel_file_directory = r'frogId_label_noise.xlsx'

# 2. read the Excel file
df = pd.read_excel(excel_file_directory, engine='openpyxl')

# 3. filter the dataframe: remove entries where the biophony is lost due to rain, wind, human_voice_or_music, or P_C_M
filtered_df = df[(df["rain"] == 4) | 
                 (df["wind"] == 4) | 
                 (df["human_voice_or_music"] == 4) | 
                 (df["P_C_M"] == 4)]

# 4. Extract the "File No" column from the filtered dataframe and convert it to a list
noisy_audio = filtered_df["File No"].tolist()

# 5. Convert each file number in the noisy_audio list to a string and append ".wav" extension
noisy_audio = [str(num) + '.wav' for num in noisy_audio]

import os
import librosa

# 6. Define the audio directory path
audio_directory = "C:\\Users\\lv228\\Downloads\\frogs\\frog_audio\\data"

# 6.1. List all the files in the directory
files = os.listdir(audio_directory)



# Define the directory path for the regions of interest
roi_directory = "C:\\Users\\lv228\\Downloads\\regions_of_interest"

# 6.3. Iterate through each file in the directory
for filename in files:
    # Process only '.wav' files and ensure they are not noisy
    if filename.endswith('.wav') and filename not in noisy_audio:  
        
        # 6.3.1. Load the audio file
        file_path = os.path.join(audio_directory, filename)
        sig, sr = librosa.load(file_path, sr=48000)
        print(filename)
        
        # 6.3.2. Extract regions of interest from the audio signal
        regions_of_interest = extract_rois_full_sig(sig, params=PARAMS_EXTRACT, display=False, verbose=False)
        
        # 6.3.3. Check if any regions of interest were extracted
        if regions_of_interest is not None:
            # Add a filename column to the regions of interest dataframe
            regions_of_interest["filename"] = filename
            fname = filename[:-4]
            output_file_path = os.path.join(roi_directory, f"{fname}.xlsx")
            
            # 6.3.4. Save each region of interest as an Excel file
            regions_of_interest.to_excel(output_file_path, index=False, engine='openpyxl')
            
            
# 7. Directory containing your Excel files
folder_path = "C:\\Users\\lv228\\Downloads\\roi"

# 7.1. List all Excel files in the directory
excel_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx') or f.endswith('.xls')]

# 7.2. Initialize an empty list to store individual dataframes
dfs = []

# 7.3. Iterate through each file, read its content and append to the dfs list
for file in excel_files:
    file_path = os.path.join(folder_path, file)
    dfs.append(pd.read_excel(file_path, engine='openpyxl'))

# 7.4. Concatenate all the individual dataframes into a single dataframe
final_df = pd.concat(dfs, ignore_index=True)


1.wav
________________________________________________________________________
10.wav
________________________________________________________________________
100.wav
________________________________________________________________________
1000.wav
________________________________________________________________________
1001.wav
________________________________________________________________________
1002.wav
________________________________________________________________________
1003.wav
________________________________________________________________________
1004.wav
________________________________________________________________________
1005.wav
________________________________________________________________________
1006.wav
________________________________________________________________________
1007.wav
________________________________________________________________________
1008.wav
________________________________________________________________________
1009.wav
_____________

In [67]:
# 8. Specify the path to the Excel file containing the labels
excel_file_path = r'C:\Users\lv228\OneDrive - University of Sussex\Desktop\frog_audio\frogId_label_noise.xlsx'

# 8.1. Load the dataframe from the specified Excel sheet
df_label = pd.read_excel(excel_file_path, sheet_name="Sheet2")

# 8.2. Display the first few rows of the dataframe to verify data
df_label.head()

,File No,Rhinella marina,Pristimantis achatinus,Hysiboas boans,Pristimantis subsigillatus,Pristimantis labiosus,Barycholos pulcher,Rhaebo haematiticus,Hypsiboas rosenbergi,Smilisca phaeota,Pristimantis walkeri,Espadarrana prosoblepon,Hyalinobatrahium aureoguttatum,Hyalinobatrachium fleischmanni,Hyalinobatrachium chirripoi,Hypsiboas picturatus,Epipedobates boulengeri,Hyloxalus toachi,Teratohyla spinosa
0,1,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0.0,0,0,0,0.0,0
1,2,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0.0,0,0,0,0.0,0
2,3,0.0,2,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0.0,0,0,0,0.0,0
3,4,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0.0,0,0,0,0.0,0
4,5,0.0,4,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0.0,0,0,0,0.0,0


In [68]:
# 9. Extract columns from the dataframe that aren't 'File No'
label_columns = [col for col in df_label.columns if col != 'File No']
print(label_columns)

# 10. Convert the labels into a numerical representation
# 10.1. Initialize an empty dictionary to store the matrix-like representation
label_to_number = {label: index + 1 for index, label in enumerate(label_columns)}

# 10.2. Display the generated label-to-number mapping
label_to_number


['Rhinella marina', 'Pristimantis achatinus', 'Hysiboas boans', 'Pristimantis subsigillatus', 'Pristimantis labiosus', 'Barycholos pulcher', 'Rhaebo haematiticus', 'Hypsiboas rosenbergi', 'Smilisca phaeota', 'Pristimantis walkeri', 'Espadarrana prosoblepon', 'Hyalinobatrahium aureoguttatum', 'Hyalinobatrachium fleischmanni', 'Hyalinobatrachium chirripoi', 'Hypsiboas picturatus', 'Epipedobates boulengeri', 'Hyloxalus toachi', 'Teratohyla spinosa']


{'Rhinella marina': 1,
 'Pristimantis achatinus': 2,
 'Hysiboas boans': 3,
 'Pristimantis subsigillatus': 4,
 'Pristimantis labiosus': 5,
 'Barycholos pulcher': 6,
 'Rhaebo haematiticus': 7,
 'Hypsiboas rosenbergi': 8,
 'Smilisca phaeota': 9,
 'Pristimantis walkeri': 10,
 'Espadarrana prosoblepon': 11,
 'Hyalinobatrahium aureoguttatum': 12,
 'Hyalinobatrachium fleischmanni': 13,
 'Hyalinobatrachium chirripoi': 14,
 'Hypsiboas picturatus': 15,
 'Epipedobates boulengeri': 16,
 'Hyloxalus toachi': 17,
 'Teratohyla spinosa': 18}

In [69]:
# 11. Initialize an empty dictionary to hold the matrix data
matrix_data={}

# 12. Convert the label-based dataframe into a matrix-like representation
# 12.1. Iterate through each row in the dataframe
for index, row in df_label.iterrows():
    filename = row['File No']
    
    # 12.2. Extract the labels from the row
    labels = [label_to_number[col] for col in label_columns if row[col] >0]
    
    # 12.3. Store the filename as the key and the labels as the value in the matrix_data dictionary
    matrix_data[ str(int(filename)) + ".wav" ] = labels

# 13. Convert the matrix_data dictionary into a pandas dataframe
matrix_df = pd.DataFrame.from_dict(matrix_data, orient='index')

# 14. Fill NaN values with 0 for better representation
matrix_df.fillna(0, inplace=True)



In [70]:
# 15. Rename the index of matrix_df to "filename" for clarity
matrix_df= matrix_df.rename_axis("filename")

# 16. Create a mask to filter rows where column 0 has non-zero values and all other columns are zero
mask = (matrix_df[0] != 0) & (matrix_df.drop(0, axis=1) == 0).all(axis=1)

# 17. Apply the mask to get the filtered dataframe
filtered_df_label = matrix_df[mask]

# 18. Define columns that need to be dropped
columns_to_drop = [1,2,3,4,5]

# 19. Drop the specified columns from the filtered dataframe in-place
filtered_df_label.drop(columns_to_drop, axis=1, inplace=True)


C:\Users\lv228\AppData\Local\Temp\ipykernel_2588\1198000240.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_label.drop(columns_to_drop, axis=1, inplace=True)


In [71]:
filtered_df_label

,0
filename,
3.wav,2.0
5.wav,2.0
9.wav,5.0
11.wav,4.0
12.wav,2.0
...,...
2692.wav,5.0
2694.wav,2.0
2695.wav,3.0


In [72]:
merged_features = final_df.merge(filtered_df_label, on='filename', how='left')
merged_features = merged_features.rename (columns = { 0 :"label"})
merged_features["label"].fillna(-1, inplace = True) 

In [73]:
merged_features

,min_y,min_x,max_y,max_x,shp_001,shp_002,shp_003,shp_004,shp_005,shp_006,...,bandwidth_y,area_xy,centroid_f,centroid_t,duration_t,bandwidth_f,area_tf,ratio_yx,filename,label
0,20,4058,96,4979,0.171679,0.192506,0.157831,0.194410,0.141017,0.154526,...,945.3125,94028,3136.71875,52.142545,10.717091,3265.62500,11753.5,0.082519,10.wav,-1.0
1,21,5435,73,5511,0.188214,0.208186,0.168917,0.214462,0.147770,0.160880,...,85.9375,7252,2878.90625,63.872000,0.884364,2234.37500,906.5,0.684211,10.wav,-1.0
2,23,5061,104,5260,0.171349,0.190693,0.156810,0.194360,0.135546,0.151551,...,171.8750,21948,3136.71875,60.101818,2.315636,3480.46875,2743.5,0.407035,10.wav,-1.0
3,28,3969,31,3997,0.163360,0.177341,0.150718,0.188606,0.204243,0.223179,...,0.0000,352,1804.68750,46.359273,0.325818,128.90625,44.0,0.107143,10.wav,-1.0
4,48,3963,95,4006,0.215527,0.237419,0.218468,0.255196,0.196837,0.212736,...,85.9375,5764,3265.62500,46.312727,0.500364,2019.53125,720.5,1.093023,10.wav,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24646,73,75,85,109,0.439661,0.465277,0.442292,0.448074,0.292769,0.292655,...,0.0000,1276,3910.15625,1.082182,0.395636,515.62500,159.5,0.352941,TE-04_1_20150723_215800.wav,-1.0
24647,73,239,83,269,0.500720,0.538685,0.500890,0.523335,0.295508,0.322609,...,0.0000,1068,3910.15625,2.967273,0.349091,429.68750,133.5,0.333333,TE-04_1_20150723_215800.wav,-1.0
24648,73,362,84,395,0.508908,0.515588,0.478962,0.487879,0.286705,0.328808,...,0.0000,1276,3910.15625,4.398545,0.384000,472.65625,159.5,0.333333,TE-04_1_20150723_215800.wav,-1.0
24649,73,5483,84,5517,0.341203,0.395338,0.370483,0.401012,0.256331,0.271625,...,0.0000,1020,3867.18750,64.011636,0.395636,472.65625,127.5,0.323529,TE-04_1_20150723_221300.wav,-1.0
